# Superdense coding

In this article we will learn about **superdense coding**: that is a method used to send 2 classical bits of information using only 1 qubit! Through this we will see the relation to notions from classical information theory such as channel capacities, and quantum analogs.

The first concept we will need is that of a communication channel. In general we will regard this informally as something that transmits information. In the real world this might be something like the under-sea cables that act as the backbone of the internet, or a telephone wire. This concept was first laid out in the Claude Shannon's "A Mathematical Theory of Communication" [1] where he defined a channel as

> merely the medium used to transmit the signal from transmitter to receiver. It may be a pair of wires, a coaxial cable, a band of radio frequencies, a beam of light, etc.

In this work Shannon also defines a channel's **capacity** as (note: there are more modern understandings of this quantity, but this simplified approach works for our purposes)

$$
C = \lim_{T\to\infty}\frac{\log N(T)}{T}
$$

where $N(T)$ is the number of signals of length $T$. This value should be thought of in units of "bits per channel use". In the case of (noiselessly) transmitting $T$ binary digits (bits), we have $N(T) = 2^T$ possible messages, and hence $\log N(T) = \log_2 (2^T) = T$. Thus channels transmitting classical bits have a capacity $C = 1$. That is if you want to transmit $n$ bits, you must send $n$ bits. No way around it!

This may seem to contradict what superdense coding stands for: that one can send two bits, using one qubit! That said, when Shannon was laying the foundation of information theory, the idea of a quantum computer had not yet been born, and hence his work is purely classical.

With the advent of this new technology, one can ask many new questions about the capacities of quantum channels (that is channels that can transmit quantum information), and even new questions about classical channels. Here are some we might think to ask:

- Can a qubit channel transmit more information than it's classical counterpart?
- Can you transmit quantum information over a classical channel?
- Can a classical channel transmit more information if the two parties share some quantum entanglement?
- Can a *quantum* channel transmit more information if the two parties share some quantum entanglement?

This last question is really the one being addressed by superdense coding. All four of these questions are interesting in their own right, however, and resources to learn more about each can be found at the end of the article.

Without further philosophizing as to the history and importance of superdense coding, lets get down to the protocol. In circuit form it can be represented as follows.

<img src="superdense.jpg" alt="superdense coding circuit diagram" width="800"/>

There are three main stages here, so let's break it down.

## Stage 1: Create shared entanglement

In this stage (colored gray in the above circuit) we will create a [Bell Pair](https://en.wikipedia.org/wiki/Bell_state) or Bell state. In particular we create the following state:

$$
    \frac{1}{\sqrt{2}}\left(|00\rangle + |11\rangle\right),
$$

but any such Bell state would theoretically work. Each such subsystem is then sent off to the respective parties: Alice and Bob. In this case Alice receives the first qubit `wire = 0`, and Bob receives the second `wire = 1`.

## Stage 2: Alice encodes her message

In this blue stage, Alice prepares her qubit to be sent to Bob so that he can receive the message $ab\in\{ 00, 01, 10, 11\}$ which Alice is sending. To this end, Alice performs a [Pauli X](https://pennylane.readthedocs.io/en/stable/code/api/pennylane.PauliX.html) gate conditioned on the second bit $b$, and a [Pauli Z](https://pennylane.readthedocs.io/en/stable/code/api/pennylane.PauliZ.html) conditioned on the first bit $a$. It is this encoding process that allows Bob to later recover the message.

Once Alice is done preparing the qubit, she must send it to Bob. This is the "channel" part of this protocol, and it's the part we analyze when thinking about how much information is exchanged here.

## Stage 3: Bob decodes the message

Finally (green), Bob is in possession of 2 qubits: one from the original creation of the Bell pair, and the other that's now been modified by Alice. Bob now performs "the opposite" of the Bell pair creation circuit and measures the two qubits. With this measurement Bob possesses the two bits Alice was sending and the protocol is complete!

---

Now we can get on with seeing how this protocol can be implemented in Pennylane. First we import `pennlylane` and set up the device we need to simulate the two qubit protocol.

In [1]:
import pennylane as qml

dev = qml.device("default.qubit", wires=2, shots=1)

Just as we broke superdense coding into three stages before, we will separate each stage into it's own function.

In [2]:
def bell_pair():
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[0, 1])
    
def encode(bits):
    a, b = bits
    if a:
        qml.PauliZ(wires=0)
    if b:
        qml.PauliX(wires=0)

def decode():
    qml.CNOT(wires=[0, 1])
    qml.Hadamard(wires=0)

We can now combine these three stages into one to complete the entire protocol.

In [3]:
@qml.qnode(dev)
def superdense_coding(bits='00'):
    bell_pair()
    
    encode(map(int, bits))
        
    decode()
    
    return qml.sample()

Let's now test this on the default case of sending the string `00`. In this case no $X$ nor $Z$ gates will be performed, and the circuit should, in theory, do nothing since we are performing

$$
(H\otimes \mathbb{1}_2)\cdot \mathtt{CNOT}^1_2 \cdot \mathtt{CNOT}^1_2 \cdot(H\otimes \mathbb{1}_2) = \left[\mathtt{CNOT}^1_2 \cdot(H\otimes \mathbb{1}_2)\right]^\dagger\cdot\left[\mathtt{CNOT}^1_2\cdot(H\otimes \mathbb{1}_2)\right] = U^\dagger U = \mathbb{1}_4.
$$

In [4]:
print(superdense_coding())

[0 0]


It works! Now let's ensure it works on the the rest of the cases.

In [5]:
print(f"sending 00: received {superdense_coding(bits='00')}")
print(f"sending 01: received {superdense_coding(bits='01')}")
print(f"sending 10: received {superdense_coding(bits='10')}")
print(f"sending 11: received {superdense_coding(bits='11')}")

sending 00: received [0 0]
sending 01: received [0 1]
sending 10: received [1 0]
sending 11: received [1 1]


So we've shown that we can transmit **two** classical bits using **one** quantum bit. But it's very rare that one only wants to send two classical bits! Most messages, especially important ones are much longer. Can this protocol be adapted?

Some care must be taken with sending messages with an odd number of bits, but otherwise we can use this protocol repeatedly to send the bits sequentially as outlined in the following code.

In [6]:
from pennylane import numpy as np

def to_string(message):
    '''
    quickly convert the output of superdense_coding to a string
    consisting of two bits `ab`
    '''
    return np.array2string(message, separator='')[1:-1]

def send_big_message(parity_indicator, bits):
    '''
    use the superdense coding protocol to send messages of arbitrary
    length. parity_indicator indicates wheter the incoming messages
    is of even or odd length:
        00 -> even length
        11 -> odd length
    '''
    indicator = to_string(superdense_coding(bits=parity_indicator))
    odd_bits_incoming = False
    if indicator == '11':
        odd_bits_incoming = True

    message = ""
    for index in range(0, len(bits), 2):
        two_bits = bits[index:index + 2]
        if len(two_bits) == 1:
            two_bits += '0' # arbitrary choice to 0 to pad
        message += to_string(superdense_coding(bits=two_bits))
    
    if odd_bits_incoming:
        return message[:-1]
    return message

This modified protocol requires sending, as part of the total message, two bits to indicate the parity of the length of the incoming message from Alice to Bob. In `send_big_message` we could naively clip the end of the message at the end of the function depending on the length of `bits`. However this doesn't account for the fact that this protocol is emulating Alice sending Bob a message, and hence Bob would not a priori know (without being told) if the message needs clipping or not. Hence we reserve the first two bits of the message to indicate this quantity so Bob can drop the last bit if needed.

In [7]:
message = '101001111'
parity = '00' if len(message) % 2 == 0 else '11'

send_big_message(parity, message)

'101001111'

That's fun and all, but we can do a lot better by providing a more concrete example. Here Alice wants to send Bob the message `dont trust the cat`. To accomplish this, Alice must first convert each word to a binary string, and send each word separately so Bob knows where each word starts and stops.

In [8]:
alices_secret = 'dont trust the cat'
words = alices_secret.split()
bin_words = [ # convert words to binary digit strings
    ''.join(format(ord(x), 'b') for x in word) for word in words
]
parities = ['00' if len(word) % 2 == 0 else '11' for word in bin_words]

bob_receives = []
for word, parity in zip(bin_words, parities):
    bob_receives.append(send_big_message(parity, word))

decoded_message = []
for word in bob_receives:
    decoded_message.append(''.join(
        [
            chr(int(word[i:i+7], 2)) # 7 binary digits/letter
            for i in range(0, len(word), 7)
        ]
    ))

print(f"Bob receives and decodes: '{' '.join(decoded_message)}'")

Bob receives and decodes: 'dont trust the cat'


In all we've provided an overview of the superdense coding protocol and shown how it transmits two bits of classical information using one quantum bit. We've also shown how to generalize this protocol to arbitrary length strings.

Since the invention of this protocol in 1970's it has been experimentally verified many times, including [recently in 2017](https://arxiv.org/abs/1609.00713) using optical fibers [2]. However, there is still the question of are quantum channel capacities larger than classical ones? This question is partially resolved in [Holevo's theorem](https://en.wikipedia.org/wiki/Holevo%27s_theorem) which states that without any shared entanglement, quantum channels do not allow more information flow through than classical ones. More information about channel capacities, and a more rigorous treatment of superdense coding (known as dense coding in this book) can be found in [3].

## References

[1] Shannon, C. (1948). A mathematical theory of communication. The Bell System Technical Journal, 27(3), 379-423.

[2] Williams, B., Sadlier, R., & Humble, T. (2017). Superdense Coding over Optical Fiber Links with Complete Bell-State Measurements. Phys. Rev. Lett., 118, 050501.

[3] Watrous, J. (2018). The Theory of Quantum Information. Cambridge University Press.

### Extras

- _Can a qubit channel transmit more information than it's classical counterpart?_
Answered by [Holevo's theorem](https://en.wikipedia.org/wiki/Holevo%27s_theorem).
- _Can you transmit quantum information over a classical channel?_
No, except in the simplest of quantum states, because an arbitrary qubit $|\psi\rangle = \alpha |0\rangle + \beta |1\rangle$ requires two complex numbers $\alpha$ and $\beta$ which could have infinite decimal expansions, and hence an infinite number of classical bits.
- _Can a classical channel transmit more information if the two parties share some quantum entanglement?_
Partially answered by [Quantum teleportation](https://en.wikipedia.org/wiki/Quantum_teleportation), which in some sense is dual to superdense coding.
- _Can a *quantum* channel transmit more information if the two parties share some quantum entanglement?_
As discussed in this article, yes, but there is much more information to be found in [3].